In [1]:
from enum import Enum

class TipoToken(Enum):  #sistema para clasificación de palabras y conectores
    O = 1
    Y = 2
    S = 3
    N = 4
    E = 5
    P = 6

In [2]:
class EstadosAnalizador(Enum):  #Sistema de guardado de Estados
    INICIO = 1
    PRECEDENTE = 2
    NEGACION = 3
    CONSECUENTE = 4
    DISYUNCION = 5
    CONJUNCION = 6
    ATOMO = 7
    ERROR1 = 8  #se esperaba un 'si', 'no' o un atomo
    ERROR2 = 9  #se esperaba un 'no' o un atomo
    ERROR3 = 10 #se esperaba un 'y', 'o', 'entonces' o un atomo
    ERROR4 = 11 #se esperaba un atomo
    FIN = 12

## Analizador Lexico
Un analizador lexico transforma una entrada en tokens

In [3]:
entrada = "Si no es un alto elfo entonces es un elfo oscuro o es un elfo del bosque"

In [4]:
tokens = entrada.lower().split()
tokens

['si',
 'no',
 'es',
 'un',
 'alto',
 'elfo',
 'entonces',
 'es',
 'un',
 'elfo',
 'oscuro',
 'o',
 'es',
 'un',
 'elfo',
 'del',
 'bosque']

In [5]:
lexemas = []
for token in tokens:
    if token == 'o':
        lexemas.append({"valor" : token , "tipo": TipoToken.O })
    elif token == 'y':
        lexemas.append({"valor" : token , "tipo": TipoToken.Y})
    elif token == 'si':
        lexemas.append({"valor" : token , "tipo": TipoToken.S})
    elif token == 'entonces':
        lexemas.append({"valor" : token , "tipo": TipoToken.E})
    elif token == 'no':
        lexemas.append({"valor" : token , "tipo": TipoToken.N})
    else :
        lexemas.append({"valor" :token , "tipo": TipoToken.P })
        
lexemas

[{'valor': 'si', 'tipo': <TipoToken.S: 3>},
 {'valor': 'no', 'tipo': <TipoToken.N: 4>},
 {'valor': 'es', 'tipo': <TipoToken.P: 6>},
 {'valor': 'un', 'tipo': <TipoToken.P: 6>},
 {'valor': 'alto', 'tipo': <TipoToken.P: 6>},
 {'valor': 'elfo', 'tipo': <TipoToken.P: 6>},
 {'valor': 'entonces', 'tipo': <TipoToken.E: 5>},
 {'valor': 'es', 'tipo': <TipoToken.P: 6>},
 {'valor': 'un', 'tipo': <TipoToken.P: 6>},
 {'valor': 'elfo', 'tipo': <TipoToken.P: 6>},
 {'valor': 'oscuro', 'tipo': <TipoToken.P: 6>},
 {'valor': 'o', 'tipo': <TipoToken.O: 1>},
 {'valor': 'es', 'tipo': <TipoToken.P: 6>},
 {'valor': 'un', 'tipo': <TipoToken.P: 6>},
 {'valor': 'elfo', 'tipo': <TipoToken.P: 6>},
 {'valor': 'del', 'tipo': <TipoToken.P: 6>},
 {'valor': 'bosque', 'tipo': <TipoToken.P: 6>}]

## Analizador Sintactico

In [7]:
EstadoActual = EstadosAnalizador.INICIO

regla = ""
pila = ""
prec = 0
pila = []
prop = ""
l_reglas = []

for lexema in lexemas:
    if EstadoActual == EstadosAnalizador.INICIO:
        if lexema['valor'] == "si":
            EstadoActual = EstadosAnalizador.PRECEDENTE
            prec = 1
        elif lexema['valor'] == "no":
            EstadoActual = EstadosAnalizador.NEGACION
            if regla:
                regla += " "
            regla += "~"
        elif lexema['tipo'] == TipoToken.P:
            EstadoActual == EstadosAnalizador.ATOMO
            pila.append(lexema['valor'])
        else:
            EstadoActual = EstadosAnalizador.ERROR1

    elif EstadoActual == EstadosAnalizador.PRECEDENTE:
        if lexema['valor'] == "no":
            EstadoActual = EstadosAnalizador.NEGACION
            if regla:
                regla += " "
            regla += "~"
        elif lexema['tipo'] == TipoToken.P:
            EstadoActual = EstadosAnalizador.ATOMO
            pila.append(lexema['valor'])
        else:
            EstadoActual = EstadosAnalizador.ERROR2

    elif EstadoActual == EstadosAnalizador.NEGACION:
        if lexema['tipo'] == TipoToken.P:
            EstadoActual = EstadosAnalizador.ATOMO
            pila.append(lexema['valor'])
        else:
            EstadoActual = EstadosAnalizador.ERROR3

    elif EstadoActual == EstadosAnalizador.CONSECUENTE:
        if lexema['tipo'] == TipoToken.P:
            EstadoActual = EstadosAnalizador.ATOMO
            pila.append(lexema['valor'])
        elif lexema['valor'] == "no":
            EstadoActual = EstadosAnalizador.NEGACION
            if regla:
                regla += " "
            regla += "~"
        else:
            EstadoActual = EstadosAnalizador.ERROR2

    elif EstadoActual == EstadosAnalizador.ATOMO:
        if lexema['valor'] == "entonces":
            EstadoActual = EstadosAnalizador.CONSECUENTE
            prop = " ".join(pila)
            if regla:
                regla += " "
            regla += prop
            if regla:
                regla += " "
            regla += "->"
            pila.clear()
            prop = ""
        elif lexema['valor'] == "y":
            EstadoActual = EstadosAnalizador.CONJUNCION
            prop = " ".join(pila)
            if regla:
                regla += " "
            regla += prop
            if regla:
                regla += " "
            regla += "&"
            pila.clear()
            prop = ""
        elif lexema['valor'] == "o":
            EstadoActual = EstadosAnalizador.DISYUNCION
            prop = " ".join(pila)
            if regla:
                regla += " "
            regla += prop
            if regla:
                regla += " "
            regla += "V"
            pila.clear()
            prop = ""
        elif lexema['tipo'] == TipoToken.P:
            EstadoActual = EstadosAnalizador.ATOMO
            pila.append(lexema['valor'])
        else:
            EstadoActual = EstadosAnalizador.ERROR3

    elif EstadoActual == EstadosAnalizador.CONJUNCION:
        if lexema['tipo'] == TipoToken.P:
            EstadoActual = EstadosAnalizador.ATOMO
            pila.append(lexema['valor'])
        elif lexema['valor'] == "no":
            EstadoActual = EstadosAnalizador.NEGACION
            if regla:
                regla += " "
            regla += "~"
        else:
            EstadoActual = EstadosAnalizador.ERROR2

    elif EstadoActual == EstadosAnalizador.DISYUNCION:
        if lexema['tipo'] == TipoToken.P:
            EstadoActual = EstadosAnalizador.ATOMO
            pila.append(lexema['valor'])
        elif lexema['valor'] == "no":
            EstadoActual == EstadosAnalizador.NEGACION
            if regla:
                regla += " "
            regla += "~"
        else:
            EstadoActual = EstadosAnalizador.ERROR4

    elif EstadoActual == EstadosAnalizador.ERROR1:
        print("se esperaba un 'si', 'no' o un atomo")
        break
    elif EstadoActual == EstadosAnalizador.ERROR2:
        print("se esperaba un 'no' o un atomo")
        break
    elif EstadoActual == EstadosAnalizador.ERROR3:
        print("se esperaba un 'y', 'o', 'entonces' o un atomo")
        break
    elif EstadoActual == EstadosAnalizador.ERROR4:
        print("se esperaba un atomo")
        break
    elif EstadoActual == EstadosAnalizador.FIN:
        break
    else:
        EstadoActual == EstadosAnalizador.FIN
        if regla:
            regla += " "
        regla += prop

if EstadoActual != (EstadosAnalizador.ERROR1 and EstadosAnalizador.ERROR2 and EstadosAnalizador.ERROR3 and EstadosAnalizador.ERROR4):
    prop = " ".join(pila)
    if regla and prop:
        regla += " " + prop
    l_reglas.append(regla)

prop = ""
pila.clear()
print(regla)
regla = ""

~ es un alto elfo -> es un elfo oscuro V es un elfo del bosque


In [54]:
print(EstadoActual)

EstadosAnalizador.ATOMO


In [8]:
with open("archivo.txt", "r") as archivo:
    lineas = archivo.readlines()
    print(lineas)

erease = "\n"
reglas = []
for linea in lineas:
    regla = linea.replace(erease, "")
    reglas.append(regla)

reglas

['Si no tiene orejas puntiagudas entonces no es un elfo\n', 'Tiene pelaje y tiene ojos de gato\n', 'Si no es un elfo alto, entonces es un elfo oscuro o es un elfo del bosque']


['Si no tiene orejas puntiagudas entonces no es un elfo',
 'Tiene pelaje y tiene ojos de gato',
 'Si no es un elfo alto, entonces es un elfo oscuro o es un elfo del bosque']